In [32]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [33]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [34]:
include("../VARs/utils.jl")
includet("../bootstrap_model.jl")

In [35]:
df = DataFrame(XLSX.readtable("../../data/usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

872×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬─────────┬────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E       │ CAPE  ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼─────────┼────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │    2.44 │ 12.52 ⋯
│ 1952-01-01 │  0.02463 │   104.771 │       23.75 │  1.41667 │ 2.42667 │ 12.36 ⋯
│ 1952-02-01 │  0.02617 │   103.585 │       23.81 │     1.42 │ 2.41333 │ 12.36 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │     2.4 │ 12.24 ⋯
│ 1952-04-01 │  0.02422 │   105.806 │       23.73 │     1.44 │    2.38 │ 12.20 ⋯
│ 1952-05-01 │  0.02384 │   106.373 │       24.38 │     1.45 │    2.36 │ 12.44 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │    2.34 │ 12.66 ⋯
│ 1952-07-01 │  0.02466 │   106.034 │       25.18 │     1.45 │ 2.34667 │ 12.67 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [36]:
freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1228,1.3903,0.1011,0.3652,-2.6994,-1.1768,-0.1044,0.2218,4.9165,0.8232,0.1442
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1239,1.3604,0.0656,0.3354,-2.5776,-1.0987,-0.1294,0.2287,4.1943,0.5297,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0       0.188888
 0.188888  1.0

In [37]:
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

835×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1955-01-01 to 2024-07-01
┌────────────┬─────────────────┬───────────────┐
│            │ Total_Stock_CPI │ Price_10Y_CPI │
├────────────┼─────────────────┼───────────────┤
│ 1955-01-01 │       0.0362793 │    -0.0101211 │
│ 1955-02-01 │      -0.0043303 │   -0.00343316 │
│ 1955-03-01 │       0.0372676 │    0.00336866 │
│ 1955-04-01 │    -0.000765351 │   -0.00320268 │
│ 1955-05-01 │       0.0594641 │    0.00504353 │
│ 1955-06-01 │       0.0697441 │    -0.0123327 │
│ 1955-07-01 │     -0.00294578 │    -0.0086779 │
│ 1955-08-01 │       0.0432164 │   0.000622423 │
│     ⋮      │        ⋮        │       ⋮       │
│ 2024-01-01 │       0.0349213 │      -0.01189 │
│ 2024-02-01 │       0.0258185 │    -0.0241356 │
│ 2024-03-01 │      -0.0140089 │     0.0036489 │
│ 2024-04-01 │       0.0231748 │    -0.0372919 │
│ 2024-05-01 │       0.0345217 │     0.0176731 │
│ 2024-06-01 │       0.0233252 │     0.0154399 │
│ 2024-07-01 │      -0.0155933 │     0.025

In [38]:
scenarios = overlapping_windows(values(data), 25*freq)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0726,0.021
5,0.0634,0.025
10,0.0586,0.0259
25,0.0606,0.0299


,Total_Stock_CPI,Price_10Y_CPI
1,0.1502,0.095
5,0.07,0.0482
10,0.0516,0.0364
25,0.0204,0.0209


,Total_Stock_CPI,Price_10Y_CPI
1,-0.5911,0.6555
5,-0.0528,0.3998
10,-0.4305,0.0251
25,0.1266,-0.4484


,Total_Stock_CPI,Price_10Y_CPI
1,0.6416,0.9705
5,-0.5279,0.3175
10,-0.9003,-0.8823
25,-0.8975,-0.5487


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.3557,-0.2426,-0.1751,-0.0246,0.085,0.1796,0.282,0.3157,0.3497
5,-0.0845,-0.0623,-0.0507,0.0087,0.0731,0.1082,0.1871,0.2037,0.2153
10,-0.0447,-0.037,-0.0297,0.0125,0.068,0.0989,0.1349,0.1397,0.1431
25,0.0249,0.0272,0.03,0.0414,0.0644,0.0726,0.0951,0.0998,0.1023


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1743,-0.1361,-0.1104,-0.0379,0.0142,0.078,0.2014,0.2781,0.2981
5,-0.0778,-0.0693,-0.0434,-0.0087,0.0196,0.0558,0.1019,0.1413,0.1578
10,-0.0477,-0.0437,-0.0261,-0.0052,0.0331,0.0534,0.0866,0.0929,0.0977
25,-0.0174,-0.0156,-0.0113,0.0135,0.0328,0.0444,0.0612,0.0629,0.0639


In [39]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.266117  0.496279  0.499561  0.516744  0.516744

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 36.05  99.0  144.0  153.0  153.0

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.12387  0.132702  0.397277  0.454945  0.468699

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 33.05  54.0  91.0  171.0  223.0